In [4]:
from sklearn.preprocessing import OneHotEncoder
from pymongo import MongoClient
from zipfile import ZipFile, ZIP_DEFLATED
from sets import Set
import numpy as np
import json, sys, os, time, re, datetime

def logTime():
    return str(datetime.datetime.now())

In [2]:
local = False
if local:
    client = MongoClient('fc8iasm01', 27017)
    plan_col = client.aca.plan
    drug_col = client.aca.drug
else:
    client = MongoClient('ec2-54-153-83-172.us-west-1.compute.amazonaws.com', 27017)
    plan_col = client.plans.plans
    drug_col = client.formularies.drugs
    prov_col = client.providers.providers
    faci_col = client.providers.facilities

all_plan = drug_col.distinct('plans.plan_id')
all_drug = drug_col.distinct('rxnorm_id')

print 'Using %s Mongo, total drug: %d, total plan: %d' %('local' if local else 'aws', len(all_drug), len(all_plan))
# client.formularies.scollection_names()
# client.providers.collection_names()

Using aws Mongo, total drug: 46206, total plan: 6035


In [5]:
print '%s: plan document: %d' %(logTime(), plan_col.count())
print '%s: drug document: %d' %(logTime(), drug_col.count())
print '%s: unique plan_id: %d' %(logTime(), len(all_plan))
print '%s: unique rxnorm_id: %d' %(logTime(), len(all_drug))

multi_plan = [1 for p in plan_col.aggregate([{"$group": {"_id":"$plan_id", "count":{"$sum":1}}}]) if p['count']>1]
print '%s: plans with multiple documents: %d' %(logTime(), sum(multi_plan))

multi_drug = [1 for p in drug_col.aggregate([{"$group": {"_id":"$rxnorm_id", "count":{"$sum":1}}}]) if p['count']>1]
print '%s: drugs with multiple documents: %d' %(logTime(), sum(multi_drug))

state_id = np.unique([i[5:7] for i in all_plan])
print '%s: states in the plan: %s' %(logTime(), ', '.join(state_id))

state = 'AK' # set to None to include all (very slow process for all)
ex_id = all_plan if not state else [i for i in all_plan if state in i]
print '%s: processing %d plans for %s ...' %(logTime(), len(ex_id), 'all' if not state else state)

2016-07-08 19:34:00.069925: plan document: 12136
2016-07-08 19:34:00.153630: drug document: 1540473
2016-07-08 19:34:00.256545: unique plan_id: 6035
2016-07-08 19:34:00.256672: unique rxnorm_id: 46206
2016-07-08 19:34:00.936182: plans with multiple documents: 1148
2016-07-08 19:35:34.493746: drugs with multiple documents: 12807
2016-07-08 19:35:34.497626: states in the plan: AK, AL, AR, AZ, CO, DE, FL, GA, HI, IA, IL, IN, KS, KY, LA, MA, ME, MI, MN, MO, MS, MT, NC, ND, NE, NH, NJ, NM, NV, OH, OK, OR, PA, SC, SD, TN, TX, UT, VA, WA, WI, WV, WY
processing 20 plans for AK ...


###Main program

In [ ]:
# ex_id = plan_col.distinct('plan_id')
# get formulary state space for all
# all_plan_states = getFormularyAllStates(plan_col, ex_id) 
# extract formulary states for each plan
# plan_feature = {p['_id']:[s in p['plan_states'] for s in all_plan_states] for p in getFormularyStatesForPlan(plan_col, ex_id)}
# get summary feature for each plan
# plan_sumstat = {p['plan']:[p['avg_copay'],p['avg_ci_rate'],p['count']] for p in getFormularyAggregate(plan_col, ex_id)}
# get all drugs covered by all plans
# all_rxnorm = drug_col.find({'plans.plan_id':{'$in':ex_id}}).distinct('rxnorm_id')
# check drug coverage for each plan
# drug_coverage = {p['plan']:[rx in p['drug'] for rx in all_rxnorm] for p in getDrugListForPlans(drug_col, ex_id)}
# get summary feature for drug
# all_drug_states = getDrugAggregateAllStates(drug_col, ex_id)
# extract drug sumstat for each plan
# drug_sumstat = {}
# for p in getDrugAggregateCountForPlans(drug_col, ex_id):
#     print '%s: processing drug coverage info for plan %s' %(logTime(), p['plan'])
#     state_count = [0]*len(all_drug_states)
#     for d in p['drug_state']:
#         state_count[all_drug_states.index(d['key'])] = d['cnt']
#     drug_sumstat[p['plan']] = state_count
# get provider states
# all_npi = prov_col.find({'plans.plan_id':{'$in':ex_id}}).distinct('npi')
# check provider coverage for each plan
# provider_coverage = {p['plan']:[npi in p['npi'] for npi in all_npi] for p in getProviderListForPlans(prov_col, ex_id)}
# # get summary feature for provider
all_provider_states = getProviderAllStates(prov_col, ex_id)

In [58]:
len(all_npi)

3493

In [11]:
for d in getDrugAggregateCountForPlans(drug_col, ex_id):
    print d,'\n'
    break

{u'plan': u'38344AK0620003', u'drug_state': [{u'cnt': 1, u'key': u'SPECIALTY|Y|Y|N'}, {u'cnt': 3, u'key': u'PREFERRED-BRAND|N|N|Y'}, {u'cnt': 10, u'key': u'NONPREFERRED-BRAND|N|Y|N'}, {u'cnt': 61, u'key': u'PREFERRED-BRAND|N|Y|N'}, {u'cnt': 2, u'key': u'PREFERRED-BRAND|Y|N|Y'}, {u'cnt': 281, u'key': u'NONPREFERRED-BRAND|N|N|N'}, {u'cnt': 418, u'key': u'PREFERRED-BRAND|N|N|N'}, {u'cnt': 4, u'key': u'SPECIALTY|Y|N|Y'}, {u'cnt': 1, u'key': u'SPECIALTY|Y|N|N'}, {u'cnt': 6, u'key': u'SPECIALTY|Y|Y|Y'}, {u'cnt': 11, u'key': u'SPECIALTY|N|Y|Y'}, {u'cnt': 10, u'key': u'NONPREFERRED-BRAND|N|N|Y'}, {u'cnt': 58, u'key': u'NONPREFERRED-BRAND|Y|N|Y'}, {u'cnt': 7, u'key': u'NONPREFERRED-BRAND|N|Y|Y'}, {u'cnt': 22, u'key': u'PREFERRED-BRAND|Y|N|N'}, {u'cnt': 156, u'key': u'NONPREFERRED-BRAND|Y|N|N'}, {u'cnt': 173, u'key': u'SPECIALTY|N|N|Y'}, {u'cnt': 243, u'key': u'SPECIALTY|N|N|N'}, {u'cnt': 25, u'key': u'NONPREFERRED-BRAND|Y|Y|N'}, {u'cnt': 115, u'key': u'GENERIC|N|Y|N'}, {u'cnt': 17, u'key': u'PR

In [13]:

test=getDrugAggregateAllStates(drug_col, ex_id)
len(test)


58

In [29]:


i=0
for p in getProviderStateForPlans(prov_col, ['38344AK0620006', '73836AK0860002']):
    print p['_id'],len(p['plan_states'])
#     break



KeyboardInterrupt: 

In [24]:
# prov_col.count()
prov_col.find_one({'facility_name':{'$exists':True}})

{u'_id': ObjectId('577606dcc421d272dcd7deaa'),
 u'accepting': u'accepting',
 u'addresses': [{u'address': u'2235 DUBOIS DR',
   u'address_2': u'',
   u'city': u'WARSAW',
   u'phone': u'2604232567',
   u'state': u'IN',
   u'zip': u'46580'}],
 u'facility_name': u'',
 u'facility_type': [],
 u'gender': u'Male',
 u'languages': [u'SPANISH'],
 u'last_updated_on': u'2016-06-18',
 u'name': {u'first': u'ALAN R', u'last': u'GILBERT MD'},
 u'npi': u'1215961677',
 u'plans': [{u'network_tier': u'PHP-SELECT',
   u'plan_id': u'50816IN0200001',
   u'plan_id_type': u'HIOS-PLAN-ID'},
  {u'network_tier': u'PHP-SELECT',
   u'plan_id': u'50816IN0200004',
   u'plan_id_type': u'HIOS-PLAN-ID'},
  {u'network_tier': u'PHP-SELECT',
   u'plan_id': u'50816IN0200007',
   u'plan_id_type': u'HIOS-PLAN-ID'},
  {u'network_tier': u'PHP-SELECT',
   u'plan_id': u'50816IN0200010',
   u'plan_id_type': u'HIOS-PLAN-ID'},
  {u'network_tier': u'PHP-SELECT',
   u'plan_id': u'50816IN0200013',
   u'plan_id_type': u'HIOS-PLAN-ID'},
 

In [27]:
faci_col.find_one() #{'npi':'1215961677'})

{u'_id': ObjectId('5775ff89c421d272dcd6877c'),
 u'addresses': [{u'address': u'4909 W DIVISION',
   u'address_2': u'',
   u'city': u'CHICAGO',
   u'phone': u'7732871200',
   u'state': u'IL',
   u'zip': u'60651'}],
 u'facility_name': u'RISHI PHARMACY INC',
 u'facility_type': [u'Pharmacy'],
 u'last_updated_on': u'2015-09-30',
 u'npi': u'1316955651',
 u'plans': [{u'name': u'Health Republic Full Access Core Platinum',
   u'network_tier': u'PREFERRED',
   u'plan_id': u'10191NJ0050003',
   u'plan_id_type': u'HIOS-PLAN-ID'},
  {u'name': u'Health Republic Full Access Core Gold',
   u'network_tier': u'PREFERRED',
   u'plan_id': u'10191NJ0050002',
   u'plan_id_type': u'HIOS-PLAN-ID'},
  {u'name': u'Health Republic Full Access Core Silver',
   u'network_tier': u'PREFERRED',
   u'plan_id': u'10191NJ0050001',
   u'plan_id_type': u'HIOS-PLAN-ID'},
  {u'name': u'Health Republic Monmouth County Community Plan Bronze',
   u'network_tier': u'PREFERRED',
   u'plan_id': u'10191NJ0180001',
   u'plan_id_type

In [6]:
client.close()
%reset